In [1]:
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import random
import tensorflow as tf
from datasets import Dataset
import tqdm as notebook_tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_test.drop(['ideology_binary','ideology_multiclass','profession','label'], axis=1, inplace=True)

In [3]:
df_test

,gender,tweet
0,male,"Ayer, en Guadalajara: feministas arrojaban áci..."
1,male,"“Chile Vamos”, o sea, la centro-derechita coba..."
2,male,"Se está tratando de silenciar que en Bolivia, ..."
3,male,"@user Lucía, lo siento pero no es así. Hebe ja..."
4,male,JxC no dio quórum para eliminar el impuesto a ...
...,...,...
43755,male,"No, si ahora resulta que todo el mundo es prim..."
43756,male,Tenemos un buen futuro si entre todos nos pone...
43757,male,Cuando el [POLITICAL_PARTY] dice la estupidez ...
43758,male,La entrevista de @user a @user hoy en El Mundo...


In [4]:
# Initialize the tokenizer
checkpoint_name= 'juan-glez29/BERTuit-gender-none'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/483k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/283k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['gender'] = label_encoder.fit_transform(df_test['gender'])

In [6]:
df_test['tweet'] = df_test['tweet'].apply(lambda x: x.replace("@user", "<usr>"))

In [7]:
def tokenize_function(example):
    return tokenizer(example['tweet'])

# Conversión del DataFrame de pandas a un Dataset de Hugging Face para pasarlo al modelo
huggingface_dataset = Dataset.from_pandas(df_test)
huggingface_dataset

Dataset({
    features: ['gender', 'tweet'],
    num_rows: 43760
})

In [8]:
# Aplicamos la tokenización a todo el dataset
tokenized_dataset = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("tweet")
test_dataset = tokenized_dataset.rename_column("gender", "labels")
test_dataset

Map:   0%|          | 0/43760 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 43760
})

In [9]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

labels = test_dataset["labels"]

macro_f1 = f1_score(labels, y_pred, average='macro')# o weighted si estan desbalceadas las clases

print(f"F1-Score:\t{macro_f1}")

F1-Score:	0.5889138102148814
